In [1]:
import pandas as pd
import numpy as np
import pickle
pd.set_option('display.max_columns',30)
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
import random
from sklearn.metrics import f1_score,roc_auc_score
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold,GroupKFold
from rfpimp import *
from xgboost import XGBClassifier
import networkx as nx #create and store graph
from node2vec import Node2Vec

/Users/crazyk/opt/anaconda3/envs/test/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
with open('train_demo.pkl','rb') as f:
    df_train_demo = pickle.load(f)
with open('train_hist.pkl','rb') as f:
    df_train_hist = pickle.load(f)

In [3]:
with open('test_demo.pkl','rb') as f:
    df_test_demo = pickle.load(f)
with open('test_hist.pkl','rb') as f:
    df_test_hist = pickle.load(f)

In [4]:
mlb = MultiLabelBinarizer()

In [5]:
DisbursalDate=df_train_demo.groupby(['ID'],as_index=False)['DisbursalDate'].min()
train_bureau=df_train_hist.merge(DisbursalDate,on=['ID'],how='left')
train_bureau_updated=train_bureau[train_bureau['DISBURSED-DT'] <= train_bureau['DisbursalDate']]
DisbursalDate=df_test_demo.groupby(['ID'],as_index=False)['DisbursalDate'].min()
test_bureau=df_test_hist.merge(DisbursalDate,on=['ID'],how='left')

In [6]:
train_bureau['train_or_test']='train'
test_bureau['train_or_test']='test'
df_bureau=train_bureau.append(test_bureau)

In [7]:
df_bureau['Time_Difference']=(df_bureau['DISBURSED-DT']-df_bureau['DisbursalDate']).dt.days
df_bureau['Time_Difference']=df_bureau['Time_Difference']/30.71
df_bureau1=df_bureau[df_bureau['Time_Difference']>=12]
df_bureau2=df_bureau[df_bureau['Time_Difference']<=0]

In [8]:
#Train
train_with_future=df_train_demo[df_train_demo['ID'].isin(df_bureau1['ID'].unique())]
train_bureau_with_future=train_bureau[train_bureau['ID'].isin(df_bureau1['ID'].unique())]
train_without_future=df_train_demo.copy()
train_bureau_without_future=train_bureau_updated.copy()

In [9]:
#Test
test_with_future=df_test_demo[df_test_demo['ID'].isin(df_bureau1['ID'].unique())]
test_bureau_with_future=test_bureau[test_bureau['ID'].isin(df_bureau1['ID'].unique())]

test_without_future=df_test_demo[df_test_demo['ID'].isin(df_bureau2['ID'].unique())]
test_bureau_without_future=test_bureau[test_bureau['ID'].isin(df_bureau2['ID'].unique())]
test_without_future=df_test_demo[~df_test_demo['ID'].isin(df_bureau1['ID'].unique())]
test_bureau_without_future=test_bureau[~test_bureau['ID'].isin(df_bureau1['ID'].unique())]

In [10]:
freq_dict = {'Half Yearly': 4, 'Monthly': 2, 'Quatrly': 3, 'BI-Monthly': 1}
instl_dict = {'Arrear': 0, 'Advance': 1}
loan_dict = {'Closed': 0, 'Active': 1}
top_up_dict = {'No Top-up Service': 0,
 ' > 48 Months': 6,
 '36-48 Months': 5,
 '24-30 Months': 3,
 '30-36 Months': 4,
 '18-24 Months': 2,
 '12-18 Months': 1}

In [11]:
df_final = train_without_future.append(test_without_future)
df_final_hist = train_bureau_without_future.append(test_bureau_without_future)

In [12]:
df_final.shape,df_final_hist.shape

((135582, 26), (280986, 27))

In [13]:
df_final[df_final['ID'] == 3]

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17,25328146,1060.0,127335,69.77,M,39.0,45257.0,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months


In [14]:
df_final_hist[df_final_hist['ID'] == 3].sort_values(by='DISBURSED-DT')

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,DisbursalDate,train_or_test
44,3,False,PRIMARY,Two-Wheeler Loan,PRB,2014-01-31,Individual,Closed,2007-09-18,2014-01-29 00:00:00,NaT,NaN,"51,796",NaN,0,NaN,NaN,0.0,NaN,"20140131,20131231,20131130,20131031,20130930,2...",0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,0,0,0,0,0,0,0,0,0,0,0,0,0",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2017-06-17,NaN
32,3,False,PRIMARY,Personal Loan,NAB,2016-12-31,Individual,Closed,2010-01-14,NaN,NaT,NaN,"30,000",815/Monthly,0,F03,NaN,0.0,Standard,"20111231,",XXX,",",",",",",48.0,2017-06-17,NaN
26,3,False,PRIMARY,Commercial Vehicle Loan,COP,2017-08-31,Joint,Closed,2012-09-28,NaN,NaT,0,"10,0,000","18,192/Monthly",0,F03,"36,576",0.0,Standard,"20170831,20170731,20170630,20170531,20170430,2...",9.5064E+58,"0,447525,459755,454879,449043,462424,493132,50...","36576,36576,36384,18192,0,0,18192,18192,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,",84.0,2017-06-17,NaN
30,3,False,PRIMARY,Commercial Vehicle Loan,COP,2018-05-31,Joint,Delinquent,2013-09-28,NaN,2018-05-31,0,"15,0,000","27,288/Monthly","7,37,749",F03,"66,914",0.0,Standard,"20180531,20180430,20180331,20180228,20180131,2...",064061062031034033DDD0330330640640330030330030...,"737749,748459,778159,804002,795732,806509,,832...","66914,77626,70038,50462,50462,43174,,33598,513...","0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",84.0,2017-06-17,NaN
42,3,False,PRIMARY,Two-Wheeler Loan,PRB,2019-12-31,Individual,Active,2013-10-24,NaN,NaT,NaN,"92,000",NaN,0,NaN,0,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,2017-06-17,NaN
49,3,False,PRIMARY,Two-Wheeler Loan,PRB,2016-04-30,Individual,Closed,2014-03-25,NaN,NaT,NaN,"44,000",NaN,0,NaN,NaN,0.0,Standard,"20150531,20150430,20150331,20150228,",XXXXXXXXXXXX,"0,0,0,3874,",",,0,0,0",",,,,,",NaN,2017-06-17,NaN
27,3,False,PRIMARY,Consumer Loan,NBF,2017-05-31,Individual,Closed,2014-08-17,2015-06-17 00:00:00,NaT,NaN,"35,700",NaN,0,NaN,0,0.0,NaN,"20150630,",XXX,",",",",",",NaN,2017-06-17,NaN
36,3,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2017-06-17,NaN,NaT,NaN,"5,19,728",NaN,"3,7,637",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",5.2021E+91,"307637,307637,307637,332006,332006,332006,3553...","0,38300,38300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2017-06-17,NaN


In [15]:
df_final_hist['ACCOUNT-STATUS'].replace({
    'Delinquent':'BadLoan',
    'Suit Filed':'BadLoan',
    'Settled':'Closed',
    'SUIT FILED (WILFUL DEFAULT)':'BadLoan',
    'WILFUL DEFAULT':'BadLoan',
    'Cancelled':'BadLoan',
    'Restructured':'Active',
    'Written Off':'BadLoan',
    'Sold/Purchased':'Closed'},inplace=True)

In [16]:
df_final_hist['ACCOUNT-STATUS'].value_counts()

Closed     191530
Active      74154
BadLoan     15302
Name: ACCOUNT-STATUS, dtype: int64

In [17]:
df_final_hist[df_final_hist['ID'] == 3].sort_values(by='DISBURSED-DT')

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,DisbursalDate,train_or_test
44,3,False,PRIMARY,Two-Wheeler Loan,PRB,2014-01-31,Individual,Closed,2007-09-18,2014-01-29 00:00:00,NaT,NaN,"51,796",NaN,0,NaN,NaN,0.0,NaN,"20140131,20131231,20131130,20131031,20130930,2...",0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,0,0,0,0,0,0,0,0,0,0,0,0,0",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2017-06-17,NaN
32,3,False,PRIMARY,Personal Loan,NAB,2016-12-31,Individual,Closed,2010-01-14,NaN,NaT,NaN,"30,000",815/Monthly,0,F03,NaN,0.0,Standard,"20111231,",XXX,",",",",",",48.0,2017-06-17,NaN
26,3,False,PRIMARY,Commercial Vehicle Loan,COP,2017-08-31,Joint,Closed,2012-09-28,NaN,NaT,0,"10,0,000","18,192/Monthly",0,F03,"36,576",0.0,Standard,"20170831,20170731,20170630,20170531,20170430,2...",9.5064E+58,"0,447525,459755,454879,449043,462424,493132,50...","36576,36576,36384,18192,0,0,18192,18192,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,",84.0,2017-06-17,NaN
30,3,False,PRIMARY,Commercial Vehicle Loan,COP,2018-05-31,Joint,BadLoan,2013-09-28,NaN,2018-05-31,0,"15,0,000","27,288/Monthly","7,37,749",F03,"66,914",0.0,Standard,"20180531,20180430,20180331,20180228,20180131,2...",064061062031034033DDD0330330640640330030330030...,"737749,748459,778159,804002,795732,806509,,832...","66914,77626,70038,50462,50462,43174,,33598,513...","0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",84.0,2017-06-17,NaN
42,3,False,PRIMARY,Two-Wheeler Loan,PRB,2019-12-31,Individual,Active,2013-10-24,NaN,NaT,NaN,"92,000",NaN,0,NaN,0,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,2017-06-17,NaN
49,3,False,PRIMARY,Two-Wheeler Loan,PRB,2016-04-30,Individual,Closed,2014-03-25,NaN,NaT,NaN,"44,000",NaN,0,NaN,NaN,0.0,Standard,"20150531,20150430,20150331,20150228,",XXXXXXXXXXXX,"0,0,0,3874,",",,0,0,0",",,,,,",NaN,2017-06-17,NaN
27,3,False,PRIMARY,Consumer Loan,NBF,2017-05-31,Individual,Closed,2014-08-17,2015-06-17 00:00:00,NaT,NaN,"35,700",NaN,0,NaN,0,0.0,NaN,"20150630,",XXX,",",",",",",NaN,2017-06-17,NaN
36,3,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2017-06-17,NaN,NaT,NaN,"5,19,728",NaN,"3,7,637",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",5.2021E+91,"307637,307637,307637,332006,332006,332006,3553...","0,38300,38300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2017-06-17,NaN


In [18]:
cols_encode = ['SEX','City','Area','BranchID','SupplierID','City','ZiPCODE']
for i in cols_encode:
    le = LabelEncoder()
    df_final[i] = df_final[i].astype(str)
    df_final[i] = df_final[i].fillna('NaN')
    le.fit(df_final[i])
    df_final[i] = le.transform(df_final[i])

In [19]:
df_final['Frequency'] = df_final['Frequency'].map(freq_dict)
df_final['InstlmentMode'] = df_final['InstlmentMode'].map(instl_dict)
df_final['LoanStatus'] = df_final['LoanStatus'].map(loan_dict)
df_final['Top-up Month'] = df_final['Top-up Month'].map(top_up_dict)

In [20]:
def evaluate_macroF1_lgb(truth, predictions):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [21]:
df_final['Days_to_mat'] = (df_final['MaturityDAte'] - df_final['DisbursalDate']).dt.days
df_final['Asset_min_Dis'] = df_final['AssetCost'] - df_final['DisbursalAmount']
df_final['Perce_EMI_Asset'] = df_final['AssetCost'] / df_final['EMI']
df_final['Perce_EMI_Dis'] = df_final['DisbursalAmount'] / df_final['EMI']
df_final['Money_Paid_Per_Month'] = df_final['DisbursalAmount']/df_final['Tenure']
df_final['Per_EMI_Monthly_income'] = df_final['MonthlyIncome']/df_final['EMI']
df_final['FOIR'] = (df_final['EMI']/df_final['MonthlyIncome'])*100

In [22]:
df_final = pd.get_dummies(df_final,columns=['ManufacturerID','State','PaymentMode'])

In [23]:
def get_quarter(date):
    if date.month >= 1 and date.month <= 3:
        return 4
    if date.month >= 4 and date.month <= 6:
        return 1
    if date.month >= 7 and date.month <= 9:
        return 2
    if date.month >= 10 and date.month <= 12:
        return 3
    
def is_quarter_start(date):
    if date.month == 1 or date.month == 4 or date.month == 7 or date.month == 10:
        return 1
    else:
        return 0
    
def is_quarter_end(date):
    if date.month == 3 or date.month == 6 or date.month == 9 or date.month == 12:
        return 1
    else:
        return 0

def is_month(date,is_start=False):
    if is_start == False:
        if date.day >= 25:
            return 1
        else:
            return 0
    if is_start == True:
        if date.day <=5:
            return 1
        else:
            return 0

In [24]:
dates = ['DisbursalDate','MaturityDAte','AuthDate']
for i in dates:
    df_final[i + '_year'] = df_final[i].apply(lambda x: x.year)
    df_final[i + '_month'] = df_final[i].apply(lambda x: x.month)
    df_final[i + '_day'] = df_final[i].apply(lambda x: x.day)
    df_final[i + '_dayofweek'] = df_final[i].apply(lambda x: x.dayofweek)
    df_final[i + '_quarter'] = df_final[i].apply(lambda x:get_quarter(x))
    df_final[i + '_is_quarter_end'] = df_final[i].apply(lambda x:is_quarter_end(x))
    df_final[i + '_is_quarter_start'] = df_final[i].apply(lambda x: is_quarter_start(x))
    df_final[i + '_is_month_start'] = df_final[i].apply(lambda x:is_month(x,is_start=True))
    df_final[i + '_is_month_false'] = df_final[i].apply(lambda x:is_month(x,is_start=False))

In [25]:
df_final_hist['TENURE'].fillna(0,inplace=True)
df_final_hist = df_final_hist[df_final_hist['DisbursalDate'].notnull()]
df_final_hist.drop_duplicates(inplace=True)
df_final_hist.reset_index(inplace=True)

In [26]:
# dates = ['DISBURSED-DT']
# for i in dates:
#     df_final_hist[i + '_year'] = df_final_hist[i].apply(lambda x: x.year)
#     df_final_hist[i + '_month'] = df_final_hist[i].apply(lambda x: x.month)
#     df_final_hist[i + '_day'] =df_final_hist[i].apply(lambda x: x.day)
#     df_final_hist[i + '_dayofweek'] = df_final_hist[i].apply(lambda x: x.dayofweek)
#     df_final_hist[i + '_quarter'] = df_final_hist[i].apply(lambda x:get_quarter(x))
#     df_final_hist[i + '_is_quarter_end'] = df_final_hist[i].apply(lambda x:is_quarter_end(x))
#     df_final_hist[i + '_is_quarter_start'] = df_final_hist[i].apply(lambda x: is_quarter_start(x))
#     df_final_hist[i + '_is_month_start'] = df_final_hist[i].apply(lambda x:is_month(x,is_start=True))
#     df_final_hist[i + '_is_month_false'] = df_final_hist[i].apply(lambda x:is_month(x,is_start=False))

In [27]:
list_cols = ['DISBURSED-AMT/HIGH CREDIT','CURRENT-BAL','CREDIT-LIMIT/SANC AMT','OVERDUE-AMT']
for i in list_cols:
    df_final_hist[i] = df_final_hist[i].fillna('0')
    df_final_hist[i] = df_final_hist[i].apply(lambda x:eval(''.join(x.split(','))))

In [28]:
df_final_hist['INSTALLMENT-AMT'] = df_final_hist['INSTALLMENT-AMT'].fillna('0')
df_final_hist['INSTALLMENT-AMT'] = df_final_hist['INSTALLMENT-AMT'].apply(lambda x:eval(''.join(x.split('/')[0].split(','))))

In [29]:
def payment_date_diff(dis,close,last_payment):
    if pd.isna(close) == False and (close.year >= 2010 and close.year <=2020):
        return (close - dis).days
    elif pd.isna(close) == True and (last_payment.year >= 2010 and last_payment.year <=2020):
        return (last_payment - dis).days
    else:
        return 0
                                    
df_final_hist['Payment_date_diff'] = df_final_hist.apply(lambda x:payment_date_diff(x['DisbursalDate'],x['CLOSE-DT'],x['LAST-PAYMENT-DATE']),axis=1)

In [30]:
df_final_hist['SELF-INDICATOR_ACCT'] = df_final_hist['SELF-INDICATOR'].astype(str) + '_' + df_final_hist['ACCT-TYPE'].astype(str)
df_final_hist['SELF-INDICATOR_CONTRI'] = df_final_hist['SELF-INDICATOR'].astype(str) + '_' + df_final_hist['CONTRIBUTOR-TYPE'].astype(str)

df_final_hist['Report_Dis_sub'] = (df_final_hist['DATE-REPORTED'] - df_final_hist['DisbursalDate']).dt.days
df_final_hist = df_final_hist.sort_values(by=['DISBURSED-DT','ID'])
df_final_hist['DISBURSED-DT'] = df_final_hist['DISBURSED-DT'].astype(str)

In [31]:
temp = df_final_hist.sort_values(by=['DISBURSED-DT','ID']).groupby('ID').agg({'ID':['count'],
    'MATCH-TYPE':['nunique'],
    'ACCT-TYPE':['nunique',','.join],
    'OWNERSHIP-IND':['nunique','count'],
    'CONTRIBUTOR-TYPE':['nunique',','.join],
    'ACCOUNT-STATUS':['nunique'],
    'DISBURSED-AMT/HIGH CREDIT':['min','max','sum','mean','std'],
    'CURRENT-BAL':['min','max','sum','mean','std'],
    'CREDIT-LIMIT/SANC AMT':['min','max','sum','mean','std'],
    'OVERDUE-AMT':['min','max','sum','mean','std'],
    'WRITE-OFF-AMT':['min','max','sum','mean','std'],
    'Payment_date_diff':['min','max','sum','mean','std'],
    'INSTALLMENT-FREQUENCY':['nunique'],
    'SELF-INDICATOR_ACCT':['nunique',','.join],
    'TENURE':['min','max','sum','mean','std'],
    'Report_Dis_sub':['min','max','sum','mean','std'],
    'SELF-INDICATOR_CONTRI':['nunique',','.join],
    'DISBURSED-DT':[','.join]})
temp.columns = ['_ID_'.join(x) for x in temp.columns]
temp.reset_index(inplace=True)
temp['DisbursalDate_ID_join'] = temp['DISBURSED-DT_ID_join'].apply(lambda x:pd.to_datetime(x.split(',')[0]))
df_final = pd.merge(df_final,temp,on='ID')

In [32]:
df_final['DisbursalDate'] = df_final['DisbursalDate'].astype(str)

In [33]:
df_final.shape

(132646, 158)

In [34]:
df_final['First_Loan_Date'] = (df_final['DisbursalDate_ID_join'] - pd.to_datetime(df_final['DisbursalDate'])).dt.days
cols_binarize = ['ACCT-TYPE_ID_join','CONTRIBUTOR-TYPE_ID_join','SELF-INDICATOR_ACCT_ID_join','SELF-INDICATOR_CONTRI_ID_join']
for i in cols_binarize:
    print(i)
    df_final[i] = df_final[i].fillna('NaN')
    df_final[i] = df_final[i].apply(lambda x:x.split(','))
    df_final = df_final.join(pd.DataFrame(mlb.fit_transform(df_final.pop(i)),columns=mlb.classes_,index=df_final.index))

ACCT-TYPE_ID_join
CONTRIBUTOR-TYPE_ID_join
SELF-INDICATOR_ACCT_ID_join
SELF-INDICATOR_CONTRI_ID_join


In [35]:
df_final_hist[df_final_hist['ID'] == 3]

,index,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,...,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,DisbursalDate,train_or_test,Payment_date_diff,SELF-INDICATOR_ACCT,SELF-INDICATOR_CONTRI,Report_Dis_sub
8,44,3,False,PRIMARY,Two-Wheeler Loan,PRB,2014-01-31,Individual,Closed,2007-09-18,2014-01-29 00:00:00,NaT,0,51796,0,...,0,0.0,NaN,"20140131,20131231,20131130,20131031,20130930,2...",0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,0,0,0,0,0,0,0,0,0,0,0,0,0",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",0.0,2017-06-17,NaN,-1235,False_Two-Wheeler Loan,False_PRB,-1233.0
5,32,3,False,PRIMARY,Personal Loan,NAB,2016-12-31,Individual,Closed,2010-01-14,NaN,NaT,0,30000,815,...,0,0.0,Standard,"20111231,",XXX,",",",",",",48.0,2017-06-17,NaN,0,False_Personal Loan,False_NAB,-168.0
2,26,3,False,PRIMARY,Commercial Vehicle Loan,COP,2017-08-31,Joint,Closed,2012-09-28,NaN,NaT,0,100000,18192,...,36576,0.0,Standard,"20170831,20170731,20170630,20170531,20170430,2...",9.5064E+58,"0,447525,459755,454879,449043,462424,493132,50...","36576,36576,36384,18192,0,0,18192,18192,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,",84.0,2017-06-17,NaN,0,False_Commercial Vehicle Loan,False_COP,75.0
4,30,3,False,PRIMARY,Commercial Vehicle Loan,COP,2018-05-31,Joint,BadLoan,2013-09-28,NaN,2018-05-31,0,150000,27288,...,66914,0.0,Standard,"20180531,20180430,20180331,20180228,20180131,2...",064061062031034033DDD0330330640640330030330030...,"737749,748459,778159,804002,795732,806509,,832...","66914,77626,70038,50462,50462,43174,,33598,513...","0,0,0,0,0,0,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",84.0,2017-06-17,NaN,348,False_Commercial Vehicle Loan,False_COP,348.0
7,42,3,False,PRIMARY,Two-Wheeler Loan,PRB,2019-12-31,Individual,Active,2013-10-24,NaN,NaT,0,92000,0,...,0,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",0.0,2017-06-17,NaN,0,False_Two-Wheeler Loan,False_PRB,927.0
9,49,3,False,PRIMARY,Two-Wheeler Loan,PRB,2016-04-30,Individual,Closed,2014-03-25,NaN,NaT,0,44000,0,...,0,0.0,Standard,"20150531,20150430,20150331,20150228,",XXXXXXXXXXXX,"0,0,0,3874,",",,0,0,0",",,,,,",0.0,2017-06-17,NaN,0,False_Two-Wheeler Loan,False_PRB,-413.0
3,27,3,False,PRIMARY,Consumer Loan,NBF,2017-05-31,Individual,Closed,2014-08-17,2015-06-17 00:00:00,NaT,0,35700,0,...,0,0.0,NaN,"20150630,",XXX,",",",",",",0.0,2017-06-17,NaN,-731,False_Consumer Loan,False_NBF,-17.0
6,36,3,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2017-06-17,NaN,NaT,0,519728,0,...,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",5.2021E+91,"307637,307637,307637,332006,332006,332006,3553...","0,38300,38300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",0.0,2017-06-17,NaN,0,True_Tractor Loan,True_NBF,958.0


In [36]:
df_final_hist = df_final_hist.sort_values(by=['ID','DISBURSED-DT'])
df_final_hist.reset_index(inplace=True,drop=True)
df_final_hist['DISBURSED-DT'] = pd.to_datetime(df_final_hist['DISBURSED-DT'])

In [37]:
df_final['DisbursalDate'] = pd.to_datetime(df_final['DisbursalDate'])

In [40]:
def check_bad_loans(demo,hist):
    demo.reset_index(inplace=True,drop=True)
    id_cust = demo['ID'][0]
    temp = hist[hist['ID'] == id_cust]
    temp.reset_index(inplace=True,drop=True)
    ref_row_idx = None
    for index, row in temp.iterrows():
        if demo['DisbursalDate'][0] == row['DISBURSED-DT']:
            ref_row_idx = index
    if ref_row_idx != None:
        row_temp = temp.iloc[ref_row_idx]
        row_date = row_temp['DISBURSED-DT']
        total_amount_disbursed = []
        for index, row in temp.iterrows():
            if (row['DISBURSED-DT'] < row_date and row['ACCOUNT-STATUS'] == 'Badloan') and row_date > row['DATE-REPORTED']:
                total_amount_disbursed.append(1)
        return sum(total_amount_disbursed)
    else:
        total_amount_disbursed = []
        for index, row in temp.iterrows():
            if (demo['DisbursalDate'][0] > row['DISBURSED-DT'] and row['ACCOUNT-STATUS'] == 'Badloan') and demo['DisbursalDate'][0] > row['DATE-REPORTED']:
                total_amount_disbursed.append(1)
        return  sum(total_amount_disbursed)
        

In [41]:
final_dict_bad = {}
for index,row in tqdm(df_final.iterrows()):
    test_demo = df_final[df_final['ID'] == row['ID']]
    test_hist=df_final_hist[df_final_hist['ID'] == row['ID']]
    emi = check_bad_loans(test_demo,test_hist)
    if emi !=0:
        final_dict_bad[row['ID']] = emi 
    else:
        final_dict_bad[row['ID']] = 0

132646it [03:37, 609.50it/s]


In [38]:
date_cols = ['DisbursalDate','MaturityDAte','AuthDate','ID','AssetID','DisbursalDate_ID_join','DISBURSED-DT_ID_join',]

In [39]:
# df_bad = pd.DataFrame.from_dict(final_dict_bad,orient='index',columns=['Bad_loans_count'])
# df_bad.reset_index(inplace=True)
# df_bad.rename(columns= {'index':'ID'},inplace=True)
# df_final = pd.merge(df_final,df_bad,on='ID')

NameError: name 'final_dict_bad' is not defined

In [ ]:
# df_final['Bad_Percent_Loans'] = df_final['Bad_loans_count']/(df_final['ID_ID_count']-1)

In [40]:
X_train = df_final[df_final['Top-up Month'].notnull()]
y_train = X_train['Top-up Month']
df_test = df_final[df_final['Top-up Month'].isnull()]
X_train.drop(['Top-up Month'],axis=1,inplace=True)
df_test.drop(['Top-up Month'],axis=1,inplace=True)

/Users/crazyk/opt/anaconda3/envs/test/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [41]:
param = {'num_leaves': 64,
         'min_data_in_leaf': 5, 
         'max_depth': -1,
         'learning_rate': 0.7,
         "boosting": "gbdt",
         "feature_fraction": 0.405,
         "lambda_l1": 1,
         "lambda_l2": 4,
         "verbosity": -1,
         'two_round': True,
         'cat_smooth': 0,
          'cat_l2': 1}

In [42]:
n_folds=5
folds=StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=22)
avg_cv = []
final_preds = np.zeros((len(df_test), 7))
final_preds_imp = pd.DataFrame()
oof_preds = np.zeros((len(X_train),7))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, y_train.values)):
        print("Fold {}".format(fold_))
        X_trn,y_trn = X_train.drop(date_cols,axis=1).iloc[trn_idx],y_train.iloc[trn_idx]
        X_val,y_val = X_train.drop(date_cols,axis=1).iloc[val_idx],y_train.iloc[val_idx]
        clf = lgb.LGBMClassifier(random_state=22,n_jobs=-1,n_estimators=5000,metric='custom')
#         clf = XGBClassifier(random_state=22,n_jobs=-1,n_estimators=2000,class_weight='balanced')
        clf.fit(X_trn, y_trn,eval_metric=evaluate_macroF1_lgb,eval_set=[(X_val,y_val)],verbose=False,early_stopping_rounds=100)
        
#         imp = importances(clf,X_val,y_val)
#         imp.reset_index(inplace=True)
#         final_preds_imp['Feature'] = X_val.columns
#         imp.rename(columns = {'Importance':f'importances{fold_}'},inplace=True)
#         final_preds_imp = pd.merge(final_preds_imp,imp,on='Feature')
        
        avg_cv.append(f1_score(y_pred=clf.predict(X_val),y_true=y_val,average='macro'))
        print(f'CV_FOLD_{fold_} : {avg_cv[fold_]}')
        final_preds += clf.predict_proba(df_test.drop(date_cols,axis=1))

final_preds = final_preds/n_folds
print(sum(avg_cv)/n_folds)

Fold 0
CV_FOLD_0 : 0.19235634693085216
Fold 1
CV_FOLD_1 : 0.20297115856300105
Fold 2
CV_FOLD_2 : 0.2086098530635276
Fold 3
CV_FOLD_3 : 0.20013713466084862
Fold 4
CV_FOLD_4 : 0.2082489827043868
0.20246469518452326


In [43]:
final_preds = final_preds/n_folds
print(sum(avg_cv)/n_folds)
avg_cv

0.20246469518452326


[0.19235634693085216,
 0.20297115856300105,
 0.2086098530635276,
 0.20013713466084862,
 0.2082489827043868]

In [44]:
# final_preds_imp['Final_importance'] = final_preds_imp[final_preds_imp.columns[1:]].sum(axis=1)

In [45]:
# final_preds_imp = final_preds_imp.sort_values(by='Final_importance',ascending=False)

In [46]:
# final_preds_imp

In [47]:
# feats = final_preds_imp[final_preds_imp['Final_importance']>0]['Feature'].values

In [48]:
len(feats)

NameError: name 'feats' is not defined

In [49]:
df_test['Top-up Month'] = [np.argmax(x) for x in final_preds]
df_test['Top-up Month'].value_counts()

<ipython-input-49-082a6d9294f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Top-up Month'] = [np.argmax(x) for x in final_preds]


0    6908
6       6
3       6
5       4
4       1
1       1
2       1
Name: Top-up Month, dtype: int64

In [50]:
top_up_dict_rev = {0:'No Top-up Service',
 6:' > 48 Months',
 5:'36-48 Months',
 3:'24-30 Months',
 4:'30-36 Months',
 2:'18-24 Months',
 1:'12-18 Months'}

In [51]:
df_test['Top-up Month'] = df_test['Top-up Month'].map(top_up_dict_rev)
df_test[['ID','Top-up Month']].to_csv('Data_Future/Past_data_preds.csv',index=False)

<ipython-input-51-f0e52932d314>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Top-up Month'] = df_test['Top-up Month'].map(top_up_dict_rev)
